<a href="https://colab.research.google.com/github/kz-yoshida/python/blob/branch1/2020_03_15_FpayForecast_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# module import
import pandas as pd

In [0]:
# GCP 認証
from google.colab import auth
auth.authenticate_user()

In [0]:
# Class定義
class BigQuery_Data: # classを定義する

# NewShop_Data生成
  def NewShops_Data(self):
    project_id = 'kouzoh-analytics-jp-prod'
    self.NewShops_Data = pd.io.gbq.read_gbq(f'''
SELECT * FROM `mercari-analytics-jp.z_issei.20202014_NewFpayrate_3month` 
''', project_id=project_id)
    self.NewShops_Data.set_index(['Partner_Size','shops_category1'],inplace=True)
    return self.NewShops_Data

# Forecast_Data生成
  def BrandList(self):
    project_id = 'kouzoh-analytics-jp-prod'
    self.BrandList = pd.io.gbq.read_gbq(f'''

## ①本日から3/31までの日数を生成
 With 
    Days AS (    
        SELECT DISTINCT
            date
          , date_diff(date,Date_sub(CURRENT_DATE('Asia/Tokyo'),Interval 1 day), day)  AS Elapsed_Days  -- 前日から各レコード日までの日数 
          , CASE WHEN Extract(DAYOFWEEK from date) = 1 AND Extract(Month from date) = 3 THEN 1 ELSE 0 END AS Sunday_Flg  -- 3月の日曜はSunday対象 
        FROM
          UNNEST(GENERATE_DATE_ARRAY(
             CURRENT_DATE('Asia/Tokyo'), '2020-03-31'            
          )) AS date
), 
## ②対象となるMiddle以上の加盟店＆Fpay情報を抽出
TargetBrandList AS (
  SELECT
        MIN(Partner_Size) AS Partner_Size
      , MAX(mst.shops_category1) AS shops_category1 
      , CAST(mst.PartnerID AS STRING) AS PartnerID
      , MAX(mst.Partner_Name) AS Partner_Name
      , mst.BrandID
      , MAX(mst.Brand_nm) AS Brand_nm
      , MIN(mst.Deploy_Date) AS Deploy_Date
      ## 現在の数値
      , COUNT(mst_sh.seq ) AS Current_Cnt_Shops  -- 現在のSHOP数
      , SUM(fpay_flag)  AS Current_Cnt_FpayShops  -- 現在のFpaySHOP数
      , ROUND( SUM(fpay_flag) /COUNT(mst_sh.seq )  ,2) AS Current_FpayRate  -- 現在のFpay率
      , COUNT(mst_sh.seq ) - SUM(fpay_flag) AS Current_Cnt_Mipay  -- 現在の未FpaySHOP数
      ## 空き間口に対して平均Dayでどれくらいの率を埋めてきたかを算出
      , IFNULL(SAFE_DIVIDE(sum( CASE WHEN Date(mst.FirstpaymentAt,'Japan') >= '2020-01-01'THEN fpay_flag ELSE NULL END),              -- 3Q以降のFpay数 
        COUNT(mst_sh.seq )  - sum(CASE WHEN Date(mst.FirstpaymentAt,'Japan') <= '2019-12-31' THEN fpay_flag ELSE NULL END)), 0)   -- 現在のShop数 - ２QendのFpay数（=3Qの空き間口数）
        / date_diff(CURRENT_DATE('Asia/Tokyo'),'2020-01-01', day) 　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　-- 3Qの日数
        AS Velocity  　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 -- DAYの平均Fpay獲得速度（空き間口に対してDAYでどらくらいの割合を獲得するか）
      ## 2Qendの数値
      , COUNT(CASE WHEN Deploy_Date <= '2019-12-31' THEN mst_sh.seq ELSE 0 END) AS SecondQ_Cnt_Shops  -- 2QendのSHOP数
      , SUM( CASE WHEN Date(mst.FirstpaymentAt,'Japan') <= '2019-12-31' THEN fpay_flag ELSE 0 END) AS SecondQ_Cnt_FpayShops  -- 2QendのFpaySHOP数      
    FROM `mercari-analytics-jp.z_issei.191013_mst_partner_v01` mst
      join `merpay-tmp-exdata-jp.connect_data.t_merpay_shop_unigue` as mst_sh
      on (mst.ShopID = mst_sh.ShopID )
    WHERE
        Partner_Size != 'Small' 
    GROUP BY 
        PartnerID
      , BrandID
    UNION ALL
    SELECT
        *
    FROM `merpay-tmp-exdata-jp.connect_data.20200229_NewShops_Deploy` 
    )
        
## ③CPN踏まえた獲得速度の調整
    SELECT
        Days.*
      , TargetBrandList.*
      , F_CPN_Flag
      , D_CPN_Flag
      ## CPN期間も含めた全期間のDAYの獲得速度
    , CASE 
           WHEN Days.date < Deploy_Date THEN 0
           WHEN Sunday_Flg = 1 THEN Velocity +0.03 　-- CPN前は従来の獲得速度を踏襲
           WHEN date < CPN_StartDate THEN Velocity 　-- CPN前は従来の獲得速度を踏襲
           WHEN date BETWEEN CPN_StartDate AND CPN_EndDate THEN 
                CASE 
       ## FEVER CPN期間の獲得速度の調整
                     WHEN F_CPN_Flag = '3.提案中'　 THEN Velocity + 0.015   -- CPN実施で3%UP×確度50%    
                     WHEN F_CPN_Flag in ( "4.合意","完了")　THEN Velocity + 0.03     -- CPN実施で3%UP×確度100%  
                     WHEN F_CPN_Flag 　in ( "0.未着手","1.アプローチ", "2.アポ取得" )　THEN Velocity + 0.003   -- CPN実施で3%UP×確度10%
       ## Debut CPN期間の獲得速度の調整
                     WHEN D_CPN_Flag = '②提案中' THEN Velocity + 0.01   -- CPN実施で2%UP×確度50%    
                     WHEN D_CPN_Flag = '③合意' 　THEN Velocity + 0.02     -- CPN実施で2%UP×確度100%  
                     WHEN D_CPN_Flag　= '①未提案' 　THEN Velocity + 0.002   -- CPN実施で2%UP×確度10%
                ELSE Velocity END 
           WHEN date >= CPN_EndDate THEN Velocity 
         ELSE Velocity END AS CPN_Include_Velocity
   ## CPN時のVelocityを集計
      , CASE WHEN D_CPN_Flag = '③合意' THEN 0.025 ELSE 0.03 END AS  CPN_Velocity   -- Debut対象はSunday加味してざっくり 0.025 後は0.03と定義
      ,  seg
      ,  SUM(Sunday_Flg) OVER (Partition by TargetBrandList.BrandID ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)   -- 前日までのSUNDAYの日のカウント
         + SUM(CASE WHEN date BETWEEN CPN_StartDate AND CPN_EndDate THEN 1 ELSE 0 END) OVER (Partition by TargetBrandList.BrandID ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)　-- 前日までのCPNの日のカウント
         AS CPN_COUNT
    FROM Days, TargetBrandList  
      LEFT OUTER JOIN `merpay-tmp-exdata-jp.connect_data.20200228_CPN_Info` CPN_Master 
      ON TargetBrandList.BrandID = CPN_Master.BrandID 

''', project_id=project_id)
    return self.BrandList


In [0]:
# BQのデータ生成
BQ1 = BigQuery_Data()
NewShops_df = BQ1.NewShops_Data()
df = BQ1.BrandList()

In [50]:
# 新規店のFpay率のParameter算出関数
def shopscategory_fpayrate(size, category):
  return NewShops_df.loc[(size),(category)][0]
  
shopscategory_fpayrate('Large','飲食') # test

0.57

In [0]:
# 日付のリストを取得
date_list = index_df['date'].dt.date.unique()
date_list

In [0]:
# Class定義(Forecas)
class Forecast: # classを定義する

# 必要なData生成
  def __init__(self):

    self.index_df =  df.set_index(['date','BrandID'],drop=False)
    # base_data(各Brandで1record)作成
    self.base_df = pd.DataFrame(df.groupby(['Partner_Size','PartnerID','Partner_Name','BrandID','Brand_nm','shops_category1']).min()[['Deploy_Date','date','Current_Cnt_Shops','Current_Cnt_FpayShops']])


In [97]:
forecast = Forecast()
index_df = forecast.index_df
base_df = forecast.base_df 

Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID          Partner_Name          BrandID            Brand_nm              shops_category1              ...             
Large        0016F0000393tcOQAQ 株式会社ファミリーマート          0016F0000393tcOQAQ 株式会社ファミリーマート          ショッピング・小売        2020-03-31  ...          NaN
             0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 飲食               2020-03-26  ...         0.57
             0016F0000396mkGQAQ 山崎製パン株式会社             0016F0000396mkGQAQ 山崎製パン株式会社             ショッピング・小売        2020-03-23  ...         0.62
             0016F0000396mxFQAQ 株式会社オートバックスセブン        0016F0000396mxFQAQ 株式会社オートバックスセブン        自動車・バイク          2020-03-31  ...         0.20
             0016F00003BifnXQAR 大東企業株式会社              0016F00003BifnXQAR 大東企業株式会社              飲食               2020-03-31  ...         0.57

[5 rows x 22 columns]

In [102]:
from datetime import timedelta,time
import datetime
## Forecastの生成

for index, item in base_df.iterrows():
  base_df.loc[index,'ontheday_fpayshops'] = item['Current_Cnt_FpayShops']   # 本日のonthedayfpayshop == 現在のFpayshop数 
  ## 既存店のロジック
  if base_df.at[index,'Deploy_Date'] <= base_df.at[index,'date']:   ## Deploydateが本日より前なら既存ロジックに当てはめる
    for t_date in date_list:
      ontheday_akimaguchi = base_df.loc[index,'Current_Cnt_Shops'] -  base_df.loc[index,'ontheday_fpayshops']  # 空き間口数 = ショップ数 - その日のFpay店舗数
      brand = index[3]
      ontheday_fpaykakutoku = ontheday_akimaguchi * index_df.xs([t_date,brand],level=['date','BrandID'])['CPN_Include_Velocity'][0].astype('float64')  # その日のfpay獲得数== その日の空き間口×velocity
      base_df.loc[index,'ontheday_fpayshops'] += ontheday_fpaykakutoku
      base_df.loc[index,t_date] = int(base_df.loc[index,'ontheday_fpayshops'])
  ## 新規店のロジック(ファミマの例外処理)
  elif index[2] == '株式会社ファミリーマート':
      for t_date in date_list:
        if t_date == datetime.date(2020, 3, 31):
          base_df.loc[index,t_date] = 3000
        else:
          base_df.loc[index,t_date] = 0
  ## 新規店のロジック
  else:
    size = index[0]       # Partner_Size取得
    category = index[5]   # category取得
    try:
      base_df.loc[index,'Avg_FpayRate'] = shopscategory_fpayrate(size,category)
      for t_date in date_list:
        delta_days = datetime.datetime.combine(t_date, time()) - base_df.loc[index,'Deploy_Date']   # 各日でdeltadaysを計算する
        if delta_days.total_seconds()/3600/24 >=0:   # deltadaysが正だったら以下のロジックを当てはめる。違ったら0にする
          base_df.loc[index,t_date] = base_df.loc[index,'Current_Cnt_Shops'] * (delta_days.total_seconds()/3600/24) * (base_df.loc[index,'Avg_FpayRate']/90)
        else:
          base_df.loc[index,t_date] = 0
    except:
      pass

base_df.tail()

Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID         Partner_Name BrandID                              Brand_nm    shops_category1              ...             
Middle       72057598308019576 東和交通株式会社     b69a43ee-e337-4313-a333-a2806920918c 東和タクシー      交通機関・サービス        2019-12-11  ...         0.15
             72057598309507622 株式会社ヒバリヤ     a894e327-d5f9-400d-8121-b9c9c37e6ddb 株式会社ヒバリヤ    ショッピング・小売        2020-02-26  ...         0.15
             72057598315877978 株式会社オレンヂ     5eaa14f7-05a1-46e6-add9-412e7e26aca0 株式会社オレンヂ    生活関連サービス         2019-10-25  ...         0.15
             72057598323402915 加茂商事株式会社     0b7d356e-665e-4f4c-9303-3f2e6d374fb2 加茂商事株式会社    スポーツ             2019-10-25  ...         0.15
             72057598323943757 株式会社クロサワ眼鏡店  f773df13-4ec8-46d3-947d-9927aa4f982f 株式会社クロサワ眼鏡店 ショッピング・小売        2020-02-20  ...         0.15

[5 rows x 22 columns]

# data確認

In [45]:
NewShops_df.head()

AVG_FpayRate
Partner_Size shops_category1              
Large        その他                      0.30
             ショッピング・小売                0.62
             スポーツ                     0.45
             ファッション                   0.43
             ペット                      0.01

In [47]:
df.head(3)

,date,Elapsed_Days,Sunday_Flg,Partner_Size,shops_category1,PartnerID,Partner_Name,BrandID,Brand_nm,Deploy_Date,Current_Cnt_Shops,Current_Cnt_FpayShops,Current_FpayRate,Current_Cnt_Mipay,Velocity,SecondQ_Cnt_Shops,SecondQ_Cnt_FpayShops,F_CPN_Flag,D_CPN_Flag,CPN_Include_Velocity,CPN_Velocity,seg,CPN_COUNT
0,2020-03-16,1,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,NaN
1,2020-03-17,2,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,0.0
2,2020-03-18,3,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,0.0


In [85]:
index_df.head(3)

,,date,Elapsed_Days,Sunday_Flg,Partner_Size,shops_category1,PartnerID,Partner_Name,BrandID,Brand_nm,Deploy_Date,Current_Cnt_Shops,Current_Cnt_FpayShops,Current_FpayRate,Current_Cnt_Mipay,Velocity,SecondQ_Cnt_Shops,SecondQ_Cnt_FpayShops,F_CPN_Flag,D_CPN_Flag,CPN_Include_Velocity,CPN_Velocity,seg,CPN_COUNT
date,BrandID,,,,,,,,,,,,,,,,,,,,,,,
2020-03-16,0016F00003943p6QAA,2020-03-16,1,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,NaN
2020-03-17,0016F00003943p6QAA,2020-03-17,2,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,0.0
2020-03-18,0016F00003943p6QAA,2020-03-18,3,0,Large,飲食,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,0016F00003943p6QAA,日本ケンタッキー・フライド・チキン株式会社,2020-03-26,600.0,0,0.0,600.0,0.0,0,0,None,None,0.0,0.03,None,0.0


In [86]:
base_df.tail(3)

Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID         Partner_Name BrandID                              Brand_nm    shops_category1              ...             
Middle       72057598315877978 株式会社オレンヂ     5eaa14f7-05a1-46e6-add9-412e7e26aca0 株式会社オレンヂ    生活関連サービス         2019-10-25  ...         0.15
             72057598323402915 加茂商事株式会社     0b7d356e-665e-4f4c-9303-3f2e6d374fb2 加茂商事株式会社    スポーツ             2019-10-25  ...         0.15
             72057598323943757 株式会社クロサワ眼鏡店  f773df13-4ec8-46d3-947d-9927aa4f982f 株式会社クロサワ眼鏡店 ショッピング・小売        2020-02-20  ...         0.15

[3 rows x 22 columns]

In [99]:
# Data確認
base_df.head(3)

Deploy_Date  ... Avg_FpayRate
Partner_Size PartnerID          Partner_Name          BrandID            Brand_nm              shops_category1              ...             
Large        0016F0000393tcOQAQ 株式会社ファミリーマート          0016F0000393tcOQAQ 株式会社ファミリーマート          ショッピング・小売        2020-03-31  ...          NaN
             0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 0016F00003943p6QAA 日本ケンタッキー・フライド・チキン株式会社 飲食               2020-03-26  ...         0.57
             0016F0000396mkGQAQ 山崎製パン株式会社             0016F0000396mkGQAQ 山崎製パン株式会社             ショッピング・小売        2020-03-23  ...         0.62

[3 rows x 22 columns]

In [103]:
# 予測値確認
base_df.sum()

Current_Cnt_Shops        165154.000000
Current_Cnt_FpayShops     92236.000000
ontheday_fpayshops        97114.299010
2020-03-16                92281.000000
2020-03-17                92389.000000
2020-03-18                92551.399000
2020-03-19                92716.798000
2020-03-20                92873.197000
2020-03-21                93036.892667
2020-03-22                94108.325444
2020-03-23                94261.758222
2020-03-24                94407.475000
2020-03-25                94574.191778
2020-03-26                94722.908556
2020-03-27                94881.955778
2020-03-28                95025.003000
2020-03-29                96431.050222
2020-03-30                96508.097444
2020-03-31                99588.144667
Avg_FpayRate                549.690000
dtype: float64

In [0]:
# spreadsheet パラメータ設定
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
auth.authenticate_user()

# Access to Google Spread Sheet
gc = gspread.authorize(GoogleCredentials.get_application_default())
# Open spreadsheet by key
wb = gc.open_by_key('1q5Ng5xX_x8TVj9160iluz3ph0LPG96X1SUk7T_ydeok')
# https://tanuhack.com/library-gspread/ : gspreadの使い方

In [0]:
##結果をSpreadsheetにExport
from gspread_dataframe import get_as_dataframe, set_with_dataframe

update_sheet = wb.worksheet("forecasttest")
# 作ったDataFrameを貼り付ける。
set_with_dataframe(update_sheet, base_df.reset_index(),resize=False, 
include_index=False)

# gspread_dataframe 参考：　https://teratail.com/questions/137584